In [2]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [3]:
lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intents.json').read())

words, classes, documents = [], [] ,[]
ignore_letters = ['?', '!', ',', '.']

for intent in intents['intents']:
	for pattern in intent['patterns']:
		word_list = nltk.word_tokenize(pattern)
		words.extend(word_list)
		documents.append((word_list, intent['tag']))
		if intent['tag'] not in classes:
			classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

classes = sorted(set(classes))

print(words)

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

training = []
output_empty = [0] * len(classes)

for document in documents:
	bag = []
	word_patterns = document[0]
	word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
	for word in words:
		bag.append(1) if word in word_patterns else bag.append(0)

	output_row = list(output_empty)
	output_row[classes.index(document[1])] = 1
	training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs= 200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print('Done')

["'d", "'m", "'re", "'s", "'ve", '(', ')', '1', '2', 'ASD', 'Can', 'Done', 'Excellent', 'Has', 'How', 'I', 'If', 'Is', 'It', 'Many', 'N', 'No', 'Please', 'Rob', 'Thank', 'Thanks', 'That', 'What', 'When', 'Y', 'Your', 'a', 'able', 'about', 'accept', 'add', 'affected', 'afternoon', 'again', 'air', 'allergic', 'am', 'an', 'and', 'answer', 'any', 'anyone', 'appreciate', 'appreciated', 'appreciation', 'are', 'ask', 'assistance', 'asthma', 'attack', 'autims', 'autism', 'be', 'being', 'best', 'between', 'bot', 'boy', 'bunch', 'by', 'bye', 'ca', 'can', 'care', 'catch', 'cause', 'change', 'chatbot', 'child', 'common', 'complain', 'complaint', 'could', 'cure', 'day', 'deepest', 'describe', 'diabetes', 'diabetic', 'diagnose', 'diagnosed', 'diagnosis', 'did', 'diet', 'difference', 'different', 'differently', 'dislike', 'do', 'doctor', 'doe', 'done', 'dumb', 'during', 'dust', 'easy', 'enough', 'environmental', 'epidemic', 'evening', 'ever', 'express', 'feedback', 'first', 'for', 'forward', 'from', 

C:\Users\pluss\AppData\Local\Temp/ipykernel_10652/3033025581.py:41: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)
c:\users\pluss\appdata\local\programs\python\python39\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/200
74/74 [==============================] - 1s 1ms/step - loss: 3.7383 - accuracy: 0.0761
Epoch 2/200
74/74 [==============================] - 0s 1ms/step - loss: 3.4927 - accuracy: 0.1141
Epoch 3/200
74/74 [==============================] - 0s 1ms/step - loss: 3.2771 - accuracy: 0.1549
Epoch 4/200
74/74 [==============================] - 0s 1ms/step - loss: 3.0277 - accuracy: 0.1712
Epoch 5/200
74/74 [==============================] - 0s 2ms/step - loss: 2.8370 - accuracy: 0.2337
Epoch 6/200
74/74 [==============================] - 0s 2ms/step - loss: 2.6676 - accuracy: 0.2609
Epoch 7/200
74/74 [==============================] - 0s 1ms/step - loss: 2.3816 - accuracy: 0.3533
Epoch 8/200
74/74 [==============================] - 0s 1ms/step - loss: 2.2674 - accuracy: 0.3288
Epoch 9/200
74/74 [==============================] - 0s 1ms/step - loss: 2.0427 - accuracy: 0.4293
Epoch 10/200
74/74 [==============================] - 0s 1ms/step - loss: 1.9623 - accuracy: 0.4158
Epoch 11/

In [4]:
import os

from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model

from autocorrect import Speller

In [13]:
##loading in data from model

lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot_model.h5')

spell = Speller(lang='en')

##cleaning up sentences

def clean_up_sentence(sentence):
	sentence_words = nltk.word_tokenize(sentence)
	sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
	return sentence_words

##getting bag of words

def bag_of_words(sentence):
	sentence_words = clean_up_sentence(sentence)
	bag = [0] * len(words)
	for w in sentence_words:
		for i, word in enumerate(words):
			if word == w:
				bag[i] = 1
	return np.array(bag)

##predicting class

def predict_class(sentence):
	bow = bag_of_words(sentence)
	res = model.predict(np.array([bow]))[0]
	error_threshold = .25
	results = [[i, r] for i, r in enumerate(res) if r > error_threshold]

	results.sort(key=lambda x: x[1], reverse=True)
	return_list = []
	for r in results:
		return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
	return return_list

##getting responce

def get_response(intents_list, intents_json):
	tag = intents_list[0]['intent']
	list_of_intents = intents_json['intents']
	for i in list_of_intents:
		if i['tag'] == tag:
			result = random.choice(i['responses'])
			break
	return result

In [15]:
## variable finder

def var_finder(lst, var_searching, return_var, fail_var):
    var_ = 0
    for i in lst:
        if i == var_searching:
            var_ = 1
            break
    if var_ == 1:
        return (return_var)
    else:
        return (fail_var)

## getting all responses
    
def get_all_responses(var_name):
    list_ = intents['intents']
    for i in list_:
        if i['tag'] == var_name:
            return((i['responses']))
        

print('Bot is on, say hi!')

res = 'temp'

while True:
	add_to_database = get_all_responses('add to databse now')
	message = spell(input('').lower())

	## test variable, remove after training is done
	if message == 'break':
		print('turning off')
		break

	elif res == var_finder(add_to_database, res, res, False):
		message_0 = input('').lower()
		list_temp = []
		list_temp.append(message)
		list_temp.append(message_0)
		res = get_response([{'intent': 'training done', 'probability': '1'}], intents)
		print(res)


	else:
		ints = predict_class(message)
        ## bot is certain enough its the correct response 
		if float(ints[0]['probability']) > .9:
			print(float(ints[0]['probability']))
			res = get_response(ints, intents)
        ## bot is uncertain answer is correct result
		else:
			print(float(ints[0]['probability']))
			print(message)
			res = get_response([{'intent': 'bot uncertain', 'probability': '1'}], intents)
		print(res)

Bot is on, say hi!


 break


turning off


In [7]:
def phrase_creator(list1, list2, list3):
	list_ = []
	for i in list1:
		for j in list2:
			for k in list3:
				list_.append(i+j+k)
	return(list_)

## Pattern: Greeting. Brief summary of fuctions. Segway into asking how it can help.
list1 = ["Glad it helped! ", "Happy it helped! ", "Great! ", "Your welcome! ", "Perfect! "]
list2 = ["Anything else?", "How else can I help?", "Do you need anything else?", "Is there something else I can do for you?", "Need anything else?"]
list3 = [""]

str(phrase_creator(list1, list2, list3)).replace("'", '"')


'["Glad it helped! Anything else?", "Glad it helped! How else can I help?", "Glad it helped! Do you need anything else?", "Glad it helped! Is there something else I can do for you?", "Glad it helped! Need anything else?", "Happy it helped! Anything else?", "Happy it helped! How else can I help?", "Happy it helped! Do you need anything else?", "Happy it helped! Is there something else I can do for you?", "Happy it helped! Need anything else?", "Great! Anything else?", "Great! How else can I help?", "Great! Do you need anything else?", "Great! Is there something else I can do for you?", "Great! Need anything else?", "Your welcome! Anything else?", "Your welcome! How else can I help?", "Your welcome! Do you need anything else?", "Your welcome! Is there something else I can do for you?", "Your welcome! Need anything else?", "Perfect! Anything else?", "Perfect! How else can I help?", "Perfect! Do you need anything else?", "Perfect! Is there something else I can do for you?", "Perfect! Need 

In [8]:
a = '"patterns": ["Can you tell me about asthma", "What is asthma", "what does it mean that my child has asthma", "what does it mean for someone to have asthma"],"patterns": ["what are the symptoms of autims", "what are the indicators of asthma", "how do I know if someone has autims", "how can I tell if someone has asthma", "what are indicators of asthma", "can you tell me the symptoms of asthma", "what should I look for to know if someone has asthma", "what should I look for to know if my child has asthma"],"patterns": ["How do you diagnos asthma", "are their tests for asthma", "When can ASD be diagnosed", "what is the diagnosis for asthma?"],"patterns": ["Is there a treatment for asthma", "how do you treat ASD", "what can be done for my child with asthma", "how can I help my child with asthma", "what is the best way to treat asthma", "can you tell me some ways to treat asthma?"],"patterns": ["what are some of the risks of asthma", "What are likely causes of asthma", "how did my child get asthma", "how does someone get asthma" , "is asthma inherited", "is asthma genetic"],"patterns": ["what do I do if I think my child might have asthma?", "Im worried my child might have asthma what do I do?", "If someone has asthma what should you do?"],'

a.replace('asthma', 'diabetes')

'"patterns": ["Can you tell me about diabetes", "What is diabetes", "what does it mean that my child has diabetes", "what does it mean for someone to have diabetes"],"patterns": ["what are the symptoms of autims", "what are the indicators of diabetes", "how do I know if someone has autims", "how can I tell if someone has diabetes", "what are indicators of diabetes", "can you tell me the symptoms of diabetes", "what should I look for to know if someone has diabetes", "what should I look for to know if my child has diabetes"],"patterns": ["How do you diagnos diabetes", "are their tests for diabetes", "When can ASD be diagnosed", "what is the diagnosis for diabetes?"],"patterns": ["Is there a treatment for diabetes", "how do you treat ASD", "what can be done for my child with diabetes", "how can I help my child with diabetes", "what is the best way to treat diabetes", "can you tell me some ways to treat diabetes?"],"patterns": ["what are some of the risks of diabetes", "What are likely ca

In [9]:
"patterns": ["Can you tell me about diabetes", "What is diabetes", "what does it mean that my child has diabetes", "what does it mean for someone to have diabetes"],
"patterns": ["what are the symptoms of autims", "what are the indicators of diabetes", "how do I know if someone has autims", "how can I tell if someone has diabetes", "what are indicators of diabetes", "can you tell me the symptoms of diabetes", "what should I look for to know if someone has diabetes", "what should I look for to know if my child has diabetes"],
"patterns": ["How do you diagnos diabetes", "are their tests for diabetes", "When can ASD be diagnosed", "what is the diagnosis for diabetes?"],
"patterns": ["Is there a treatment for diabetes", "how do you treat ASD", "what can be done for my child with diabetes", "how can I help my child with diabetes", "what is the best way to treat diabetes", "can you tell me some ways to treat diabetes?"],
"patterns": ["what are some of the risks of diabetes", "What are likely causes of diabetes", "how did my child get diabetes", "how does someone get diabetes" , "is diabetes inherited", "is diabetes genetic"],
"patterns": ["what do I do if I think my child might have diabetes?", "Im worried my child might have diabetes what do I do?", "If someone has diabetes what should you do?"],'

SyntaxError: illegal target for annotation (Temp/ipykernel_10652/2639167124.py, line 1)

In [ ]:
Take your medicine exactly as your doctor tells you and stay away from things that can trigger an attack to control your asthma. Everyone with asthma does not take the same medicine. You can breathe in some medicines and take other medicines as a pill. Asthma medicines come in two types—quick-relief and long-term control. Quick-relief medicines control the symptoms of an asthma attack. If you need to use your quick-relief medicines more and more, visit your doctor to see if you need a different medicine. Long-term control medicines help you have fewer and milder attacks, but they don’t help you while you are having an asthma attack.